# Asad Rizvi
# 19i-1778
# i191778@nu.edu.pk


# Algorithm (General)

-  **  BIGRAMS  **:


- Tokenize corpus into words
- Generate bigrams
- Count frequency of each bigram
- remove redundant bigrams, punctuations, empty rows
- create list of initial words of sentences for picking the first word
- select number of lines for the poetry (10)
- select number of words in a verse (6--8)
- for each verse:
  - choose starting word from the list of intial words
  - generate next word using bigrams (most probable)




-  **  TRIGRAMS  **:


- Tokenize corpus into words
- Generate trigrams
- Count frequency of each trigram
- remove redundant trigrams, punctuations, empty rows
- create list of initial words(2) of sentences for picking the first two words
- select number of lines for the poetry (10)
- select number of words in a verse (6--8)
- for each verse:
  - choose starting word from the list of intial words
  - generate next word using trigrams (most probable)




-  **  BACKWARD BIGRAMS **:


- Tokenize corpus into words
- Reverse the list of words
- Generate bigrams
- Count frequency of each bigram
- remove redundant bigrams, punctuations, empty rows
- create list of initial words of sentences for picking the first word
- select number of lines for the poetry (10)
- select number of words in a verse (6--8)
- for each verse:
  - choose starting word from the list of intial words
  - generate next word using bigrams (most probable)





-  **  BIDIRECTIONAL  **:


- first word of each verse for both forward bigrams and backward bigrams is same
- select number of lines for the poetry (10)
- select number of words in a verse
- for each verse:
  - choose starting word from the list of intial words
  - generate next word using bigrams (forward and the backward)





In [ ]:
import spacy
import re
import os
import pandas as pd
import random
# pd.set_option("display.max_rows", None, "display.max_columns", None)

In [ ]:
# ///////  generate BiGram list  //////////////

def generateBiGram(trainingWords):

    bigram_list = []

    for i in range(0,len(trainingWords)-1):
        first = trainingWords[i]
        second = trainingWords[i+1]
        total = [first, second]
        bigram_list.append(total)

    return bigram_list




# ///////  generate TriGram list  //////////////

def generateTriGram(trainingWords):

    trigram_list = []

    for i in range(0,len(trainingWords)-2):
        first = trainingWords[i]
        second = trainingWords[i+1]
        third = trainingWords[i+2]
        total = [first, second, third]
        trigram_list.append(total)

    return trigram_list


In [ ]:
# ///////  Loading the corpus  //////////////

trainingCorpus = open("poetry.txt").read()




# ///////  Removing empty lines  //////////////

trainingCorpusNoEmptyLines = os.linesep.join([s for s in trainingCorpus.splitlines() if s])





# ///////  Splitting the corpus  //////////////

trainingWords = re.split(r'[\s\n]', trainingCorpusNoEmptyLines)





# ///////  Finding Unique Words  //////////////

trainingWordsUnique = []
for word in trainingWords:
    if word not in trainingWordsUnique:
        trainingWordsUnique.append(word)





# ///////  Storing count of unique words  //////////////

uniqueWordsCount = pd.DataFrame(columns=['Word', 'Count'])


for i in trainingWordsUnique:
    count=0

    for j in (trainingWords):
        if i==j:
            count+=1

    uniqueWordsCount = uniqueWordsCount.append({'Word':i , 'Count':count}, ignore_index=True)

uniqueWordsCount

,Word,Count
0,گلوں,1
1,میں,562
2,رنگ,40
3,بھرے,1
4,باد,12
...,...,...
4839,رزق,2
4840,اچھی,1
4841,کوتاہی,1
4842,حالت,1


# Bigrams

In [ ]:
# ///////  Generating bigrams  /////////

bigramDataFrame = pd.DataFrame(columns=['Word', 'Coming After', 'Count'])

bigram_list = generateBiGram(trainingWords)

for i in bigram_list:

    count=0
    for j in range(0,len(trainingWords)-1):
        if (trainingWords[j]==i[0] and trainingWords[j+1]==i[1]):
            count+=1

    bigramDataFrame = bigramDataFrame.append({'Word':i[1], 'Coming After':i[0], 'Count':count}, ignore_index=True)

# dropping duplicates, puntuations, empty rows
bigramDataFrame.drop_duplicates(inplace=True)
bigramDataFrame = bigramDataFrame[(bigramDataFrame['Word']!="")]
bigramDataFrame = bigramDataFrame[(bigramDataFrame['Coming After']!="")]
bigramDataFrame = bigramDataFrame[(bigramDataFrame['Word']!="’’")]
bigramDataFrame = bigramDataFrame[(bigramDataFrame['Coming After']!="’’")]
display(bigramDataFrame)

,Word,Coming After,Count
0,میں,گلوں,1
1,رنگ,میں,1
2,بھرے,رنگ,1
3,باد,بھرے,1
4,نوبہار,باد,1
...,...,...,...
24051,اپنی,آپ,1
24052,حالات,اپنی,1
24053,کے,حالات,1
24054,بدلنے,کے,1


In [ ]:
# ///////  Finding Starting Word of Sentences for bigrams  /////////

firstWordsDataFrame = pd.DataFrame(columns=['Word', 'Count'])

first_words_of_sentences = []
for i in trainingCorpusNoEmptyLines.split('\n'):
    first_words_of_sentences.append(i.split(' ')[0])

for i in first_words_of_sentences:
    count=0
    for j in first_words_of_sentences:
        if i==j:
            count+=1

    firstWordsDataFrame = firstWordsDataFrame.append({'Word':i , 'Count':count}, ignore_index=True)

firstWordsDataFrame.drop_duplicates(inplace=True)

firstWordsDataFrame = firstWordsDataFrame[(firstWordsDataFrame['Count'] >= 20) & (firstWordsDataFrame['Word']!="")]
firstWordsDataFrame.reset_index(drop=True, inplace=True)
firstWordsDataFrame


,Word,Count
0,جو,38
1,اس,25
2,ہر,25
3,نہ,52
4,دل,58
5,ہم,41
6,وہ,58
7,جس,20
8,کیا,30
9,تو,32


In [ ]:
# ///////  Generating Poetry through bigrams /////////

sentence = []
firstWords = []
poetryBigram = []
firstWordsBigram = []

verseLength = random.randint(6,8)

for lines in range(0,10):

#     selecting first word
    while(1):
        firstWordIndex = random.randint(0,firstWordsDataFrame.shape[0]-1)
        if firstWordIndex not in firstWords:
            break

    firstWords.append(firstWordIndex)

#     print(firstWordsDataFrame.iloc[firstWordIndex])
    sentence.append(firstWordsDataFrame.iloc[firstWordIndex]['Word'])
    firstWordsBigram.append(firstWordsDataFrame.iloc[firstWordIndex]['Word'])

# selecting rest of the words
    for i in range(0,verseLength-1):
        next_word = bigramDataFrame[bigramDataFrame['Coming After']==sentence[i]].sort_values('Count',ascending=False).head(1)['Word'].values[0]
        sentence.append(next_word)

    poetryBigram.append(sentence)
    sentence = []


for i in range(0,len(poetryBigram)):
    print(" ".join(poetryBigram[i]))
    if i%2!=0:
        print()


اس کی طرح عیش نیام سے گزر گیا
دل کا نہ ہوا ہے کہ میں ہے

کیا ہے کہ میں ہے کہ میں ہے
کوئی کشمکش اندوہِ عشق و دل کا نہ

یہ بات پر نہیں آتی ہے کہ میں
نہ ہوا ہے کہ میں ہے کہ میں

تو کیا ہے کہ میں ہے کہ میں
جو تو کیا ہے کہ میں ہے کہ

کہ میں ہے کہ میں ہے کہ میں
ہم نے کیا ہے کہ میں ہے کہ



# Trigrams

In [ ]:
trigramDataFrame = pd.DataFrame(columns=['Word', 'Coming After 2', 'Coming After 1', 'Count'])

trigram_list = generateTriGram(trainingWords)

for i in trigram_list:

    count=0
    for j in range(0,len(trainingWords)-2):
        if (trainingWords[j]==i[0] and trainingWords[j+1]==i[1] and trainingWords[j+2]==i[2]):
            count+=1

    trigramDataFrame = trigramDataFrame.append({'Word':i[2], 'Coming After 2':i[1],'Coming After 1':i[0], 'Count':count}, ignore_index=True)



In [ ]:
trigramDataFrame.drop_duplicates(inplace=True)
trigramDataFrame = trigramDataFrame[(trigramDataFrame['Word']!="")]
trigramDataFrame = trigramDataFrame[(trigramDataFrame['Coming After 2']!="")]
trigramDataFrame = trigramDataFrame[(trigramDataFrame['Coming After 1']!="")]
trigramDataFrame = trigramDataFrame[(trigramDataFrame['Word']!="’’")]
trigramDataFrame = trigramDataFrame[(trigramDataFrame['Coming After 1']!="’’")]
trigramDataFrame = trigramDataFrame[(trigramDataFrame['Coming After 2']!="’’")]
display(trigramDataFrame.sort_values('Count', ascending=False).head(20))

,Word,Coming After 2,Coming After 1,Count
6384,کو,اُس,کہیں,19
329,لیا,دیکھ,کے,12
9504,ہو,نہ,کیوں,11
9503,نہ,کیوں,ہی,11
7268,ہیں,دیکھتے,کو,8
18281,ہیں,بھی,اور,8
11334,نہیں,خاک,میں,8
15495,تو,ہوتا,نہ,7
5097,ہوا,نہ,تسلی,7
608,بھر,رات,رہی,7


In [ ]:
# ///////  Finding Starting Words(2) of Senteces for trrgrams  /////////

first2WordsDataFrame = pd.DataFrame(columns=['Word 2', 'Word 1', 'Count'])

first_2words_of_sentences = []

for i in trainingCorpusNoEmptyLines.split('\n'):
    try:
        first_2words_of_sentences.append([ i.split(' ')[0],i.split(' ')[1] ])
    except:
        continue

# for i in first_2words_of_sentences:
#     print(i)

for i in first_2words_of_sentences:
    count=0
    for j in first_2words_of_sentences:
        if i==j:
            count+=1

    first2WordsDataFrame = first2WordsDataFrame.append({'Word 2':i[1] , 'Word 1':i[0]  , 'Count':count}, ignore_index=True)

first2WordsDataFrame.drop_duplicates(inplace=True)

first2WordsDataFrame = first2WordsDataFrame[ (first2WordsDataFrame['Count'] >= 3) &(first2WordsDataFrame['Word 1']!="") & (first2WordsDataFrame['Word 2']!="")]
first2WordsDataFrame.reset_index(drop=True, inplace=True)
first2WordsDataFrame.sort_values('Count', ascending=False)

,Word 2,Word 1,Count
24,نے,ہم,10
2,میں,دل,8
23,نے,میں,7
40,ہو,نہ,6
5,ہوں,میں,6
...,...,...,...
59,تصویر,قیس,3
60,نے,زخم,3
61,بھی,تیر,3
62,تری,جو,3


In [ ]:
# ///////  Generating Poetry through trigrams /////////

sentence = []
firstWords = []
poetryTrigram = []
firstWordsTrigram = []

verseLength = random.randint(6,8)

for lines in range(0,10):

#     selecting first word
    while(1):
        firstWordIndex = random.randint(0,first2WordsDataFrame.shape[0]-2)
        if firstWordIndex not in firstWords:
            break


    firstWords.append(firstWordIndex)

    sentence.append(first2WordsDataFrame.iloc[firstWordIndex]['Word 1'])
    sentence.append(first2WordsDataFrame.iloc[firstWordIndex]['Word 2'])


# selecting rest of the words
    for i in range(0,verseLength-2):

        next_word = (trigramDataFrame[(trigramDataFrame['Coming After 1']==sentence[i]) & (trigramDataFrame['Coming After 2']==sentence[i+1])].sort_values('Count',ascending=False).head(1)['Word'].values[0])
#         next_word = trigramDataFrame[ trigramDataFrame[(trigramDataFrame['Coming After 1']==sentence[i]) & (trigramDataFrame['Coming After 2']==sentence[i+1])  ]].sort_values('Count',ascending=False).head(1)['Word'].values[0]
        sentence.append(next_word)

#         break

#     break

    poetryTrigram.append(sentence)
    sentence = []


for i in range(0,len(poetryTrigram)):
    print(" ".join(poetryTrigram[i]))
    if i%2!=0:
        print()


چمن میں لالہ دکھاتا پھرتا ہے داغ اپنا
ان کو اپنا بنا کے دیکھ لیا دل

وہ خاک کہ ایک جوہرِ تیغِ قضا کہیں
کس کا جنون دید تمنا شکار تھا آئینہ

اپنے پہ اعتماد ہے غیر از نمود کچھ
تھا زندگی میں مرگ کا کھٹکا لگا ہوا

کبھی تو نہ توڑ سکتا اگر استوار ہوتا
ہے یہ وقت ہے شگفتنِ گل ہائے ناز

ہم نے اپنا دل خوں کیا ہوا پایا
آپ سے کوئی پوچھے تم نے کیا مزا



# Backward Bigram

In [ ]:
# ///////  Reversing corpus /////////

trainingWordsBackward = trainingWords
trainingWordsBackward.reverse()



# ///////  Generating bigrams for reversed corpus  /////////

bigramReverseDataFrame = pd.DataFrame(columns=['Word', 'Coming After', 'Count'])

bigramReverse_list = generateBiGram(trainingWordsBackward)

for i in bigramReverse_list:

    count=0
    for j in range(0,len(trainingWordsBackward)-1):
        if (trainingWordsBackward[j]==i[0] and trainingWordsBackward[j+1]==i[1]):
            count+=1

    bigramReverseDataFrame = bigramReverseDataFrame.append({'Word':i[1], 'Coming After':i[0], 'Count':count}, ignore_index=True)




# dropping duplicates, puntuations, empty rows
bigramReverseDataFrame.drop_duplicates(inplace=True)
bigramReverseDataFrame = bigramReverseDataFrame[(bigramReverseDataFrame['Word']!="")]
bigramReverseDataFrame = bigramReverseDataFrame[(bigramReverseDataFrame['Coming After']!="")]
bigramReverseDataFrame = bigramReverseDataFrame[(bigramReverseDataFrame['Word']!="’’")]
bigramReverseDataFrame = bigramReverseDataFrame[(bigramReverseDataFrame['Coming After']!="’’")]
display(bigramReverseDataFrame)

,Word,Coming After,Count
0,بدلنے,کا,1
1,کے,بدلنے,1
2,حالات,کے,1
3,اپنی,حالات,1
4,آپ,اپنی,1
...,...,...,...
24051,باد,نوبہار,1
24052,بھرے,باد,1
24053,رنگ,بھرے,1
24054,میں,رنگ,1


In [ ]:
# ///////  Generating Poetry through bigrams reversed /////////

sentence = []
firstWords = []
poetryBigramReverse = []
firstWordsBigramReverse = []

verseLength = random.randint(6,8)

for lines in range(0,10):

#     selecting first word
    while(1):
        firstWordIndex = random.randint(0,firstWordsDataFrame.shape[0]-1)
        if firstWordIndex not in firstWords:
            break

    firstWords.append(firstWordIndex)

#     print(firstWordsDataFrame.iloc[firstWordIndex])
    sentence.append(firstWordsDataFrame.iloc[firstWordIndex]['Word'])
    firstWordsBigramReverse.append(firstWordsDataFrame.iloc[firstWordIndex]['Word'])

# selecting rest of the words
    for i in range(0,verseLength-1):
        next_word = bigramReverseDataFrame[bigramReverseDataFrame['Coming After']==sentence[i]].sort_values('Count',ascending=False).head(1)['Word'].values[0]
        sentence.append(next_word)

    poetryBigramReverse.append(sentence)
    sentence = []


for i in range(0,len(poetryBigramReverse)):
    print(" ".join(poetryBigramReverse[i]))
    if i%2!=0:
        print()


کوئی سے مجھ ڈبویا ہوتا نہ کیوں
نہ کیوں ہی تو ہے کیا تو

ہر حال پوچھ نہ کیوں ہی تو
اس اب ہے کیا تو ہے کیا

جو ہے کیا تو ہے کیا تو
کس ہے کیا تو ہے کیا تو

میں ہوں میں ہوں میں ہوں میں
وہ ہے کیا تو ہے کیا تو

جس ہے کیا تو ہے کیا تو
کیا تو ہے کیا تو ہے کیا



# Bidirectional

Both the bigram model and the backWard bigram model should take the same input. In our case, the same input would be the initial starting words of the sentences.

In [ ]:
# generating through backward bigram using the same starting words

sentence = []
firstWords = []
poetryBigramReverse = []
firstWordsBigramReverse = []

verseLength = random.randint(6,8)

iterator = 0

for lines in range(0,10):

    sentence.append(firstWordsBigram[iterator])
    firstWordsBigramReverse.append(firstWordsDataFrame.iloc[firstWordIndex]['Word'])

# selecting rest of the words
    for i in range(0,verseLength-1):
        next_word = bigramReverseDataFrame[bigramReverseDataFrame['Coming After']==sentence[i]].sort_values('Count',ascending=False).head(1)['Word'].values[0]
        sentence.append(next_word)

    poetryBigramReverse.append(sentence)
    sentence = []

    iterator+=1

for i in range(0,len(poetryBigramReverse)):
    print(" ".join(poetryBigramReverse[i]))
    if i%2!=0:
        print()


اس اب ہے کیا تو ہے کیا
دل و دل و دل و دل

کیا تو ہے کیا تو ہے کیا
کوئی سے مجھ ڈبویا ہوتا نہ کیوں

یہ ہے کیا تو ہے کیا تو
نہ کیوں ہی تو ہے کیا تو

تو ہے کیا تو ہے کیا تو
جو ہے کیا تو ہے کیا تو

کہ ہے کیا تو ہے کیا تو
ہم ہے کیا تو ہے کیا تو



# Both forward bigram and backward bigram take the same input. (Same starting words of the sentences). Output is seen in the above cells



# Observations:

-- Backward bigram model produces better rhyming stanzas and does so more frequently

-- Generally, among al the methods tried, trigrams produce the most realistic output.